In [6]:
import pandas as pd
import time

In [7]:
df_cvs = pd.read_csv('data/resume.csv')

# Take 50% relevant and 50% irrelevant CVs (30 CVs total)
# Filter CVs by category
relevant_cvs_it = df_cvs[df_cvs['Category'] == 'INFORMATION-TECHNOLOGY'].sample(n=50, random_state=42)
irrelevant_cvs_it = df_cvs[df_cvs['Category'] != 'INFORMATION-TECHNOLOGY'].sample(n=50, random_state=42)

relevant_cvs_sales = df_cvs[df_cvs['Category'] == 'SALES'].sample(n=50, random_state=42)
irrelevant_cvs_sales = df_cvs[df_cvs['Category'] != 'SALES'].sample(n=50, random_state=42)

relevant_cvs_hr = df_cvs[df_cvs['Category'] == 'HR'].sample(n=50, random_state=42)
irrelevant_cvs_hr = df_cvs[df_cvs['Category'] != 'HR'].sample(n=50, random_state=42)

# get only 5 of each
relevant_cvs_it = relevant_cvs_it.sample(n=5, random_state=5)
irrelevant_cvs_it = irrelevant_cvs_it.sample(n=5, random_state=5)
relevant_cvs_sales = relevant_cvs_sales.sample(n=5, random_state=5)
irrelevant_cvs_sales = irrelevant_cvs_sales.sample(n=5, random_state=5)
relevant_cvs_hr = relevant_cvs_hr.sample(n=5, random_state=5)
irrelevant_cvs_hr = irrelevant_cvs_hr.sample(n=5, random_state=5)

# Combine into one dataset
df_cvs_it = pd.concat([relevant_cvs_it, irrelevant_cvs_it], ignore_index=True)
df_cvs_sales = pd.concat([relevant_cvs_sales, irrelevant_cvs_sales], ignore_index=True)
df_cvs_hr = pd.concat([relevant_cvs_hr, irrelevant_cvs_hr], ignore_index=True)

df_llm1_it = pd.read_csv('data/processed/base_results/cvs_it_test_scores_base_LLM.csv')
df_llm1_hr = pd.read_csv('data/processed/base_results/cvs_hr_test_scores_base_LLM.csv')
df_llm1_sales = pd.read_csv('data/processed/base_results/cvs_sales_test_scores_base_LLM.csv')
df_llm2_it = pd.read_csv('data/processed/base_results/cvs_it_test_scores_advanced_LLM.csv')
df_llm2_hr = pd.read_csv('data/processed/base_results/cvs_hr_test_scores_advanced_LLM.csv')
df_llm2_sales = pd.read_csv('data/processed/base_results/cvs_sales_test_scores_advanced_LLM.csv')

df_llm1_it = df_llm1_it[df_llm1_it["ID"].isin(df_cvs_it["ID"])]
df_llm1_hr = df_llm1_hr[df_llm1_hr["ID"].isin(df_cvs_it["ID"])]
df_llm1_sales = df_llm1_sales[df_llm1_sales["ID"].isin(df_cvs_it["ID"])]
df_llm2_it = df_llm2_it[df_llm2_it["ID"].isin(df_cvs_it["ID"])]
df_llm2_hr = df_llm2_hr[df_llm2_hr["ID"].isin(df_cvs_it["ID"])]
df_llm2_sales = df_llm2_sales[df_llm2_sales["ID"].isin(df_cvs_it["ID"])]

In [8]:
# read in job posts
# we agreed on 1 from each of the following categories for now: HR SALES INFORMATION-TECHNOLOGY
df_jobposts = pd.read_csv('data/processed/jobposts.csv')

categories = ['INFORMATION-TECHNOLOGY', 'SALES', 'HR']

df_filtered = df_jobposts[df_jobposts['category'].isin(categories)]

# Sample 1 job postings per category
df_sampled = df_filtered.groupby('category', group_keys=False).sample(n=1, random_state=42)
df_sampled = df_sampled[['category', 'job_description']]

# Split into separate DataFrames
df_it = df_sampled[df_sampled['category'] == 'INFORMATION-TECHNOLOGY']
df_sales = df_sampled[df_sampled['category'] == 'SALES']
df_hr = df_sampled[df_sampled['category'] == 'HR']

# Job Post:

In [9]:

"""
    You will need to consider the following factors:
    ### Criteria for Evaluating CVs
    * **Educational Qualifications**: 
        + Degree and field of study
        + Institution and location
        + Date of completion
        + Any relevant certifications
    * **Work Experience**: 
        + Job titles and roles
        + Company names and locations
        + Dates of employment
        + Brief descriptions of job responsibilities
    * **Certifications**: relevant certifications and licenses
    * **Achievements**: notable achievements and awards
    * **Relevant Projects**: relevant projects and contributions
    * **Skills**: 
        + **Necessary Skills**: skills required to perform the job
        + **Trainable Skills**: skills that can be learned or developed
        + **Additional Skills**: skills that might be helpful but not essential
    * **Personal Attributes and Values**: 
        + Soft skills such as communication, teamwork, and problem-solving
        + Personal values such as adaptability, reliability, and integrity
    * **Red Flags**: 
        + Job hopping
        + Stagnant jobs and roles
        + Excessive mistakes in the resume
    * **Job Requirements**: how well the candidate's skills and experience match the job requirements
    * **Company Culture**: how well the candidate's values and personality align with the company culture
"""
""
print(df_it.iloc[0]["job_description"])


ATR International, Inc. is an IT staffing leader and places a wide range of technical professionals at many of the most innovative companies. We are currently recruiting for the following position for one of our clients. If this position is not a fit for you, click here to send us your resume and our recruiters will find one that is. Position Information: Position title: Infrastructure Project Manager w/ system integration project experienceLocation: Salt Lake City, UT Compensation: DEO Contact Information: Kathy Van: KathyV@atr1.comJob Description:Responsible for leading project teams and managing all activities in a project life-cycle (initiation planning, executing/controlling, and closing) associated with high visibility projects that are long-term, across one or more lines of business and carry a substantial impact to the relevant businesses. Interacts with senior leadership and major stakeholders to establish strategic plans and objectives for programs or projects. Consults with 

In [21]:
reviewer_scores = []  # To store scores as integers

for i, row in df_cvs_it.iterrows():
    print("\n========================")
    print(f"Resume #{i + 1}:\n")
    print(row["Resume_str"])
    print("\n========================")
    time.sleep(0.1)
    time.sleep(0.1)

    while True:
        try:
            score = int(input(f"""Your score (0–10): """))
            if 0 <= score <= 10:
                reviewer_scores.append(score)
                break
            else:
                print("⚠️ Please enter a number between 0 and 10.")
        except ValueError:
            print("⚠️ Invalid input. Please enter an integer between 0 and 10.")

print("\n✅ Review complete. Here are your scores:")
print(reviewer_scores)


Resume #1:

         INFORMATION TECHNOLOGY CONSULTANT, MANAGING MEMBER           Summary    A versatile, analytic IT Specialist with a proven record of success within large institutions as well as entrepreneurial organizations. Thrives on challenge and solves problems with creativity and persistence. A data-driven team leader skilled in both producing and communicating results.      Experience      Information Technology Consultant, Managing Member    January 2017   to   Current     Company Name   －   City  ,   State       Worked with product designers and product managers to design user interactions in applications.    Envisioned inspired new products, features and flows.   Answered user inquiries regarding computer software and hardware operation.    Installed and performed repairs to hardware, software and peripheral equipment, following design and installation specifications.  Set up equipment for employee use.   Conducted computer diagnostics to investigate and resolve problems 

KeyboardInterrupt: Interrupted by user

In [20]:
import numpy as np
from scipy.stats import spearmanr
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import ndcg_score

# Data
llm1=[]
llm2=[]
for i, row in df_cvs_it.iterrows():
    llm1.append(df_llm1_it[df_llm1_it["ID"] == row["ID"]].iloc[0]["score"])
    llm2.append(df_llm2_it[df_llm1_it["ID"] == row["ID"]].iloc[0]["score"])

# Convert to numpy arrays
y_true = np.array(reviewer_scores)
y_pred1 = np.array(llm1)
y_pred2 = np.array(llm2)

# 1. Spearman Correlation
spearman1, _ = spearmanr(y_true, y_pred1)
spearman2, _ = spearmanr(y_true, y_pred2)

# 2. Mean Absolute Error
mae1 = mean_absolute_error(y_true, y_pred1)
mae2 = mean_absolute_error(y_true, y_pred2)

# 3. NDCG@10
# ndcg_score expects a 2D array for true and predicted relevance
ndcg1 = ndcg_score([y_true], [y_pred1])
ndcg2 = ndcg_score([y_true], [y_pred2])

# Print results
print("=== Evaluation Results ===")
print(f"Spearman Correlation - LLM 1: {spearman1:.3f}, LLM 2: {spearman2:.3f}")
print(f"MAE                  - LLM 1: {mae1:.3f}, LLM 2: {mae2:.3f}")
print(f"NDCG@10              - LLM 1: {ndcg1:.3f}, LLM 2: {ndcg2:.3f}")

=== Evaluation Results ===
Spearman Correlation - LLM 1: 0.863, LLM 2: 0.872
MAE                  - LLM 1: 1.200, LLM 2: 1.300
NDCG@10              - LLM 1: 0.983, LLM 2: 0.976


✅ 1. Spearman’s Rank Correlation
    Measures similarity in ranking order between each LLM and the human reviewer.
    Higher is better (1.0 is perfect).

✅ 2. MAE (Mean Absolute Error)
    Measures how far the LLM’s scores are from human scores on average.
    Lower is better.

✅ 3. NDCG@10
    Measures the ranking quality using graded relevance (0–10 reviewer scores).
    Focuses on quality of top-10 ranked candidates.
    Higher is better (1.0 is perfect).